In [ ]:
import os
import glob
import random
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np
# Cosine Annealing için artık ReduceLROnPlateau'ya gerek yok
from torch.optim.lr_scheduler import CosineAnnealingLR 
import timm

# ==================== AYARLAR ====================
MODEL_ADI = 'tf_efficientnetv2_m.in21k'
MODEL_PATH = f"{MODEL_ADI}_cosine_scheduler_adcverili.pth" # Yeni model adı

# Hiperparametreler
BATCH_SIZE = 16
EPOCHS = 100 # Cosine scheduler ile biraz daha uzun eğitmek genellikle faydalıdır
PATIENCE = 10 # Sabrı da artırabiliriz
LR_BACKBONE = 2e-5 # Biraz daha agresif bir başlangıç
LR_HEAD = 2e-3
WEIGHT_DECAY = 1e-3

# Veri Yolları ve Diğer Ayarlar (Aynı)
BASE_DATA_DIR = "/home/comp5/ARTEK/SYZ_25/SYZ_25_Egitim/IKINCI_GOREV/yeni_veri_imbalancli_splitted"
TRAIN_DATA_ROOT = os.path.join(BASE_DATA_DIR, "train")
VAL_DATA_ROOT = os.path.join(BASE_DATA_DIR, "val")
CLASS_NAMES = ["HiperakutAkut", "NormalKronik", "Subakut"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 42
random.seed(RANDOM_SEED); np.random.seed(RANDOM_SEED); torch.manual_seed(RANDOM_SEED)
# =================================================

# ==== Dataset, build_sequences, create_model (Aynı) ====
class MRDataset(Dataset):
    def __init__(self, sequences, transform): self.sequences = sequences; self.transform = transform
    def __len__(self): return len(self.sequences)
    def __getitem__(self, idx):
        img_path, label = self.sequences[idx]
        image = self.transform(Image.open(img_path[0]).convert("RGB"))
        return image, torch.tensor(label)
def build_sequences(data_root):
    sequences = []; class_to_idx = {cls: i for i, cls in enumerate(CLASS_NAMES)}
    for cls in CLASS_NAMES:
        img_paths = sorted(glob.glob(os.path.join(data_root, cls, "*.png")))
        for path in img_paths: sequences.append(([path], class_to_idx[cls]))
    return sequences
def create_model(model_name, num_classes):
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    return model.to(DEVICE)

# ==== Eğitim Fonksiyonu (Cosine Scheduler ile Güncellendi) ====
def train():
    temp_model = create_model(MODEL_ADI, len(CLASS_NAMES)); data_config = timm.data.resolve_data_config({}, model=temp_model)
    img_size = data_config['input_size'][1]; del temp_model
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)), transforms.TrivialAugmentWide(), transforms.ToTensor(),
        transforms.Normalize(mean=data_config['mean'], std=data_config['std'])
    ])
    train_seq = build_sequences(TRAIN_DATA_ROOT); val_seq = build_sequences(VAL_DATA_ROOT)
    labels = [label for _, label in train_seq]; class_counts = np.bincount(labels, minlength=len(CLASS_NAMES))
    class_weights_tensor = torch.tensor([sum(class_counts) / c if c > 0 else 0 for c in class_counts], dtype=torch.float).to(DEVICE)
    train_dataset = MRDataset(train_seq, transform); val_dataset = MRDataset(val_seq, transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)
    
    model = create_model(MODEL_ADI, len(CLASS_NAMES))
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)

    # Discriminative LR (Aynı)
    head_params = list(model.classifier.parameters())
    backbone_params = [p for p in model.parameters() if not any(p is hp for hp in head_params)]
    param_groups = [{'params': backbone_params, 'lr': LR_BACKBONE}, {'params': head_params, 'lr': LR_HEAD}]
    optimizer = torch.optim.AdamW(param_groups, weight_decay=WEIGHT_DECAY)
    
    # YENİ SCHEDULER
    scheduler = CosineAnnealingLR(optimizer, T_max=len(train_loader) * EPOCHS, eta_min=1e-7)

    best_val_loss = float('inf')
    patience_counter = 0
    
    print(f"\n🚀 {MODEL_ADI} modeli ile Cosine Annealing Scheduler ile eğitim başlıyor...\n")

    for epoch in range(1, EPOCHS + 1):
        model.train(); train_loss, train_acc = 0, 0
        
        # tqdm'ı döngünün dışına alarak daha temiz bir ilerleme çubuğu
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
        for x, y in pbar:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            # Her adımdan sonra scheduler.step() çağırıyoruz
            scheduler.step()
            
            train_acc += (out.argmax(1) == y).sum().item()
            train_loss += loss.item() * x.size(0)
            pbar.set_postfix(lr=optimizer.param_groups[0]['lr'], loss=loss.item())

        train_acc /= len(train_dataset); train_loss /= len(train_dataset)
        
        # Doğrulama döngüsü...
        model.eval(); val_loss, val_acc = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE); out = model(x); loss = criterion(out, y)
                val_acc += (out.argmax(1) == y).sum().item(); val_loss += loss.item() * x.size(0)
        val_acc /= len(val_dataset); val_loss /= len(val_dataset)
        
        print(f"📊 Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss; torch.save(model.state_dict(), MODEL_PATH); print(f"   ✨ Model kaydedildi. En iyi Val Loss: {best_val_loss:.4f}"); patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE: print(f"⛔ Early stopping at epoch {epoch}."); break
            
    print(f"✅ Eğitim tamamlandı. En iyi model ({MODEL_ADI}): {MODEL_PATH}")

if __name__ == "__main__":
    train()

In [ ]:
import os
import glob
import random
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import timm
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

MODEL_ADI = '/home/comp5/ARTEK/SYZ_25/SYZ_25_Egitim/IKINCI_GOREV/veri_png_splitle_egitilen/tf_efficientnetv2_m.in21k'
MODEL_PATH = f"{MODEL_ADI}_cosine_scheduler_best_olacak.pth" # Yeni model adı

# Hiperparametreler
BATCH_SIZE = 4 # Test için BATCH_SIZE'ı artırabilirsiniz, bu hızı etkiler

# Veri Yolları
BASE_DATA_DIR = "/home/comp5/ARTEK/SYZ_25/SYZ_25_Egitim/IKINCI_GOREV/VERI/veri_png_split"
TEST_DATA_ROOT = os.path.join(BASE_DATA_DIR, "test")

# Diğer Ayarlar
CLASS_NAMES = ["HiperakutAkut", "NormalKronik", "Subakut"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 42
random.seed(RANDOM_SEED); np.random.seed(RANDOM_SEED); torch.manual_seed(RANDOM_SEED)

class MRDataset(Dataset):
    def __init__(self, sequences, transform):
        self.sequences = sequences
        self.transform = transform
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        img_path, label = self.sequences[idx]
        image = self.transform(Image.open(img_path[0]).convert("RGB"))
        return image, torch.tensor(label)

def build_sequences(data_root):
    sequences = []
    class_to_idx = {cls: i for i, cls in enumerate(CLASS_NAMES)}
    for cls in CLASS_NAMES:
        img_paths = sorted(glob.glob(os.path.join(data_root, cls, "*.png")))
        for path in img_paths:
            sequences.append(([path], class_to_idx[cls]))
    return sequences

def create_model(model_name, num_classes):
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    return model.to(DEVICE)

def test():
    temp_model = create_model(MODEL_ADI, len(CLASS_NAMES))
    data_config = timm.data.resolve_data_config({}, model=temp_model)
    img_size = data_config['input_size'][1]
    del temp_model

    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=data_config['mean'], std=data_config['std'])
    ])

    print(f"📂 Test verisi yükleniyor: {TEST_DATA_ROOT}")
    test_seq = build_sequences(TEST_DATA_ROOT)
    test_dataset = MRDataset(test_seq, transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    
    print(f"🧪 Toplam {len(test_dataset)} test örneği bulundu.")

    model = create_model(MODEL_ADI, len(CLASS_NAMES))
    print(f"💾 Model ağırlıkları yükleniyor: {MODEL_PATH}")
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval() # Modeli DEĞERLENDİRME MODUNA AL (çok önemli!)

    all_labels = []
    all_preds = []

    print("🚀 Değerlendirme başlıyor...")
    with torch.no_grad(): # Gradient hesaplamayı kapat
        for images, labels in tqdm(test_loader, desc="Test Ediliyor"):
            images = images.to(DEVICE)
            
            outputs = model(images)
            predictions = torch.argmax(outputs, 1)

            all_labels.extend(labels.numpy())
            all_preds.extend(predictions.cpu().numpy())

    # Metrikleri Hesapla
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=CLASS_NAMES, zero_division=0)

    # Sonuçları Yazdır
    print("\n" + "="*40)
    print("📊 DETAYLI TEST SONUÇLARI")
    print("="*40)
    print(f"Doğruluk (Accuracy): {accuracy:.4f}")
    print(f"Makro F1 Skoru:      {macro_f1:.4f}")
    print("\nSınıflandırma Raporu (Classification Report):\n")
    print(report)
    print("="*40)

    # Karışıklık Matrisini (Confusion Matrix) Görselleştir
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    plt.xlabel('Tahmin Edilen Sınıf (Predicted Class)', fontsize=12)
    plt.ylabel('Gerçek Sınıf (True Class)', fontsize=12)
    plt.title('Karışıklık Matrisi (Confusion Matrix)', fontsize=14)
    plt.show()

if __name__ == "__main__":
    test()